<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text to Image
### Method: CLIP + Siren
This model combines an image classifier called **CLIP** from *OpenAI* and an image generator called **SIREN.**

The process works like this:
1.  The **SIREN** model at first generates a completely random image
2.  The **CLIP** image classifier looks at the image and compares it with the given text caption
3.  Using this feedback the weights of the **SIREN** model are adjusted slightly to better approximate the image caption
4.  These steps are repeated until we stop it manually
---
In order to run click the [Colab] link. And then select *Runtime -> Run all* from the menu bar.

[colab]: https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb

# Parameters

In [4]:
text = 'An illustration of a black and white cat'  #@param {type: "string"}
args = '--lower_bound_cutout 0.5 --image-width=384 --iterations=1000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=123'  #@param {type: "string"}

parameters = {
    "text": text,
    "args": args
}

# Code

In [5]:
!pip install deep-daze --upgrade
!pip install stomp.py imageio-ffmpeg asyncio websockets varname --upgrade

     |████████████████████████████████| 1.4MB 8.9MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 3.3MB 34.6MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
     |████████████████████████████████| 92kB 12.9MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.1-cp37-none-any.whl size=41573 sha256=630c45563aa644dca3c8011404146282a530ab0081cedda1bca4978d1b6a315f
  Stored in directory: /root/.cache/pip/wheels/ae/73/c7/9056e14b04919e5c262fe80b54133b1a88d73683d05d7ac65c
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=db2c9dd8a4602fc6f9af2ce93bb6e7eebe1e86fd54a876a42f7785409646896d
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built ftfy fire
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling im

In [7]:

#parameter_names = [varname(p) for p in parameters]


In [8]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 
import re
flatten = itertools.chain.from_iterable

TEXT="A black and white technical illustration of merging a cat and a human through biohacking."
DEFAULT_PARAMS="--lower_bound_cutout 0.5 --image-width=384 --iterations=1000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=123"
GITHUB_REPO="voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb"
GITHUB_REPO_CLEAN = re.sub('[^0-9a-zA-Z]+', '_', GITHUB_REPO)
print(GITHUB_REPO_CLEAN)
conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)
conn.subscribe(destination=f'/queue/deep-daze', id=1, ack='auto')

def run(text, params=DEFAULT_PARAMS, watch_media_paths=["./*.jpg","./*.mp4"]):
  print("Running",text)
  for to_remove in watch_media_paths:
    for filepath in glob(to_remove):
      print("Deleting",filepath,"before starting.")
      os.remove(filepath)

  cmd = f'imagine {params} "{text}"'
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)

    headers = { 
        "params": params,
        "text": text
    }
    conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files

    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          filename = os.path.basename(added)
          mimetype, _ = guess_type(filename)
          print("Adding:", added,"type",mimetype)
          
          with open(added,"rb") as media_file:
            added_media = media_file.read()

            base64_encoded_media = f"data:{mimetype};base64,{b64encode(added_media).decode('ascii')}"
          
          headers = {
              **headers,         
              "type": mimetype,
              "filename": filename
              }
          conn.send(body=base64_encoded_media, headers=headers, destination='/topic/colabMediaOut')

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))


class MyListener(stomp.ConnectionListener):
    def __init__(self, conn,run):
        self.conn = conn
        self.run = run

    def on_error(self, frame):
        print('received an error "%s"' % frame.body)

    def on_message(self, frame):
        print('received a message "%s"' % frame.body)
        self.run(frame.body)
        print('processed message')

    def on_disconnected(self):
        print('disconnected')
        #connect_and_subscribe(self.conn)

conn.set_listener('', MyListener(conn,run))

#run(TEXT)
#conn.disconnect()
sleep(1000000)


voodoohop_colabasaservice_blob_master_colabs_deep_daze_ipynb
received a message "A scientific illustration of a penis penetrating an asshole in black and white"
Running A scientific illustration of a penis penetrating an asshole in black and white
epochs:   0%|          | 0/2 [00:00<?, ?it/s]
                                             
Adding: ./A_scientific_illustration_of_a_penis_penetrating_an_asshole_in_black_and_whit.jpg type image/jpeg
loss: -22.43:   0%|          | 1/1000 [00:01<15:25,  1.08it/s]
                                             
Adding: ./A_scientific_illustration_of_a_penis_penetrating_an_asshole_in_black_and_whit.000001.jpg type image/jpeg
loss: -24.54:   0%|          | 3/1000 [00:03<15:13,  1.09it/s]
                                             
Adding: ./A_scientific_illustration_of_a_penis_penetrating_an_asshole_in_black_and_whit.000002.jpg type image/jpeg
loss: -26.58:   0%|          | 5/1000 [00:05<15:06,  1.10it/s]
                                         

KeyboardInterrupt: ignored

In [ ]:
conn.disconnect()

In [ ]:
!npm install -g npm@latest
!npm install -g --force --unsafe-perms node-gyp node-pre-gyp bcrypt ipfs


In [10]:
!nohup jsipfs daemon &
!sleep 10

nohup: appending output to 'nohup.out'


In [16]:

!jsipfs swarm connect /ip4/18.157.173.110/tcp/4002/p2p/QmWxrFJqurnyedzJ5rzpgycTqSikJ7kcy7eHmqYnndKhwr

!curl -X POST "http://18.157.173.110:5002/api/v0/pin/add?arg=QmSbe593T3ZWmtPpjXppz8do3Xy749TLUBQDaMVWK7Ceib"

connect /ip4/18.157.173.110/tcp/4002/p2p/QmWxrFJqurnyedzJ5rzpgycTqSikJ7kcy7eHmqYnndKhwr success
